# Load modules and define functions


In [1]:
##Import stuff
import pandas as pd
import geopandas
import pyproj
import matplotlib
import numpy as np
from osgeo import gdal, ogr, osr
import sqlite3
import random
from shapely import wkt
import glob
import gdal
import numpy as np
from sklearn.model_selection import train_test_split
import os

In [2]:
def get_boundaries(image):
    '''
    Bestimmen der Bildgrenzen
    '''
    import gdal
    src = gdal.Open(image)
    ulx, xres, xskew, uly, yskew, yres = src.GetGeoTransform()
    lrx = ulx + (src.RasterXSize * xres)
    lry = uly + (src.RasterYSize * yres)
    return ulx, xres, uly, yres, lrx, lry

# Preparation of mosaics

## Cutout of training images

This can happen either
- to cut a mosaic to be classified into small pieces
- to try an object classification system 
- to train the super resolution model 

This needs to be done for each mosaic that was picked in QGIS

### Variables for cutting

In [33]:
os.chdir('/home/peter/git/retina_seafloor')
#mosaic_folder = './stone_dataset/GeoTIFS/Kriegers_Flak/mosaics_for_train/' #needs to end with /
mosaic_folder = '/home/peter/git/retina_seafloor/stone_dataset/GeoTIFS/Kriegers_Flak/mosaics_for_test/'#needs to end with /
input_mosaics = ['steintest2_25cm_upscaled_by_SR_25res.tif']
#input_mosaics = ['1_BS_HOM_GR.png', '2_BS_HOM_GR.png' ]
#target_folder = './tiles/Object_Detect/100m_25cm/'
target_folder = '/home/peter/git/retina_seafloor/applydata/kw_test/25cm_upscaled_SR/'
tile_size = '20 20'
overlap_pixels = '2'
shape_index_name = 'tiles.shape'

### Do the cutting


In [34]:
for element in input_mosaics:
    # Therefore, this is the workaround with a call to the command line, which requires that gdal is installed on the PC. It is also available on google colab
    cmd = 'gdal_retile.py -ps ' + tile_size + ' -targetDir ' + target_folder + \
            ' ' + mosaic_folder + element
    print("gdal command for tiling images")
    print(cmd)
    os.system(cmd)

gdal command for tiling images
gdal_retile.py -ps 20 20 -targetDir /home/peter/git/retina_seafloor/applydata/kw_test/25cm_upscaled_SR/ /home/peter/git/retina_seafloor/stone_dataset/GeoTIFS/Kriegers_Flak/mosaics_for_test/steintest2_25cm_upscaled_by_SR_25res.tif


In [32]:
#Create tfw files if needed
#funktioniert nicht für gedrehte bilder

import osgeo.osr as osr
import os
import glob
import sys

def generate_tfw(path, scale_factor = 4, gen_prj='prj'):
    for infile in glob.glob(os.path.join(path, '*.tif')):
        src = gdal.Open(infile)
        xform = src.GetGeoTransform()

        if gen_prj == 'prj':
            src_srs = osr.SpatialReference()
            src_srs.ImportFromWkt(src.GetProjection())
            src_srs.MorphToESRI()
            src_wkt = src_srs.ExportToWkt()

            prj = open(os.path.splitext(infile)[0] + '.tif.png.tif.prj', 'wt')
            prj.write(src_wkt)
            prj.close()

        src = None
        corr_1 = xform[1] / scale_factor
        corr_5 = xform[5] / scale_factor
        edit1=xform[0]+corr_1/2
        edit2=xform[3]+corr_5/2

        tfw = open(os.path.splitext(infile)[0] + '.tif.png.tif.tfw', 'wt')
        tfw.write("%0.8f\n" % corr_1)
        tfw.write("%0.8f\n" % xform[2])
        tfw.write("%0.8f\n" % xform[4])
        tfw.write("%0.8f\n" % corr_5)
        tfw.write("%0.8f\n" % edit1)
        tfw.write("%0.8f\n" % edit2)
        tfw.close()
        
generate_tfw('/home/peter/git/SRGAN-tensorflow/data/')

## Create test/train csv files with class annotation and link to the exported tiles. 

This will read QGIS sqslite databases with coordinates stored in WKT formats


### Variables for class annotation

In [19]:
os.chdir('/home/peter/git/retina_seafloor')

image_folder = './tiles/Object_Detect/25m_25cm_multires/' #end with /   folder with tiles 

image_type = '.tif'

databases = ['./stone_dataset/Annotations/v2pickedstone.sqlite',
            './stone_dataset/Annotations/v2empty.sqlite'] #sqlite with picked objects

table_names = ['v2pickedstone',
              'v2empty'] # table name to use in the sqlite file, normally the filename without sqlite at the end

buffers = ['no',
          'yes']  # yes when points are imported, create recangular buffer around dots. 

buffer_value = 1  #buffer size in m

should_we_add_class = 'yes' # add a class name to the data?

class_names = ['stone',
              'empty']   # empy exmaples for retinanet-training go by ''empty''

appendix_for_annotation_file = '100x100_25cm_stones_empty_3band'

export_for_ = 'retinanet'

### Code for variable assignment

In [20]:
assert len(databases) == len(class_names) == len(table_names)

# einlesen der sqlite datenbank mit den gepickten rechtecken oder punkten

data = [] #make empty result list
for i, (database, class_name, table_name, should_we_buffer) in enumerate(zip(databases, class_names, table_names, buffers)):
    print('Working on: ' , database, class_name, table_name)
    cn = sqlite3.connect(database)
    sql_query = "SELECT * FROM " + table_name
    vector_df = pd.read_sql_query(sql_query, cn)

    #Create geopandas
    vector_df['Coordinates'] = vector_df['WKT_GEOMETRY'].apply(wkt.loads)
    vector_gdf = geopandas.GeoDataFrame(vector_df, geometry='Coordinates')
    # get image list
    image_list = glob.glob(image_folder + '*' + image_type)


    results = []
    for image in image_list:
        ulx, xres, uly, yres, lrx, lry = get_boundaries(image)
        results.append([image, ulx, uly, lrx, lry, xres, yres])

    raster_df = pd.DataFrame(results, columns = ['image', 'ulx', 'uly', 'lrx', 'lry', 'xres', 'yres'])

    # add emtpy imagename columns to vector gdf
    vector_gdf['image_path'] = ''

    if should_we_add_class == 'yes':
        vector_gdf['classname'] = class_name

    ##Compare the two data frames
    for row in vector_gdf.itertuples():
        #bounds return (minx, miny, maxx, maxy)
        idx = row.Index
        minx = float(row.Coordinates.bounds[0])
        miny = float(row.Coordinates.bounds[1])
        maxx = float(row.Coordinates.bounds[2])
        maxy = float(row.Coordinates.bounds[3])

        if should_we_buffer == 'yes':
            #this allows "rebuffer"
            mean_x = (float(minx) + float(maxx)) /2
            mean_y = (float(maxy) + float(miny))/2

            minx = mean_x - buffer_value/2
            maxx = mean_x + buffer_value/2
            miny = mean_y - buffer_value/2
            maxy = mean_y + buffer_value/2


        image_exists = []
        image_exists = raster_df[(raster_df.ulx.values < minx) & (raster_df.uly.values > maxy) & (raster_df.lrx.values > maxx) & (raster_df.lry.values < miny )]
        if not image_exists.empty:
            for image in image_exists.iterrows():
                # get unqie list of objects/images and calculate pixels
                image_xmin = image[1].ulx
                image_ymax = image[1].uly

                image_res_y = image[1].yres
                image_res_x = image[1].xres


                pixel_ymin = np.abs(int((image_ymax - maxy) / image_res_y))
                pixel_ymax = np.abs(int((image_ymax - miny) / image_res_y))
                pixel_xmin = np.abs(int((minx - image_xmin) / image_res_x))
                pixel_xmax = np.abs(int((maxx - image_xmin) / image_res_x))



                if pixel_ymin == pixel_ymax:
                    print('Image resolution to low for picked data, skipping entry (ymin == ymax)')
                    continue
                if pixel_xmin == pixel_xmax:
                    print('Image resolution to low for picked data, skipping entry (xmin == xmax')
                    continue

                data.append(dict({'classname': row.classname, 'imagename' : image[1].image, 'minx' : minx, 'miny' : miny, 'maxx' : maxx, 'maxy' : maxy, 'pixelxmax': pixel_xmax, 'pixelxmin' : pixel_xmin, 'pixelymax' : pixel_ymax, 'pixelymin': pixel_ymin}))


#Make annotated csv filenames
df = pd.DataFrame(data)




if export_for_ == 'tfrecord':
# Reformat for usage with Tensorflow API and no longer Keras Retinanet
  df['width'] = df.maxx - df.minx
  df['height'] = df.maxy - df.miny
  columns = ['imagename', 'width', 'height', 'classname', 'pixelxmin', 'pixelymin', 'pixelxmax', 'pixelymax']

  # For tfrecords we have to remove the paths, because we have to give the image dir directly 
  df.imagename = df.imagename.apply(lambda x: os.path.split(x)[1])


  # Export Test and Train sets
  train, test = train_test_split(df, test_size=0.2)
  train.to_csv(appendix_for_annotation_file + 'train.csv', header = None, index = None, sep = ',', columns = columns)
  test.to_csv(appendix_for_annotation_file+ 'test.csv', header = None, index = None, sep = ',',  columns = columns)




mask = (df['classname'] == 'empty')
df.at[mask, 'minx'] = 99999

if export_for_ == 'retinanet':
# Reformat for usage with Tensorflow API and no longer Keras Retinanet



  columns = ['imagename', 'pixelxmin', 'pixelymin', 'pixelxmax', 'pixelymax', 'classname']

  # Export Test and Train sets
  train, test = train_test_split(df, test_size=0.2)
  train.to_csv(appendix_for_annotation_file + 'train.csv', header = None, index = None, sep = ',', columns = columns)
  test.to_csv(appendix_for_annotation_file+ 'test.csv', header = None, index = None, sep = ',',  columns = columns)

  #resolve empty classes ..only for retinannet 
  columns = ['imagename', 'pixelxmin', 'pixelymin', 'pixelxmax', 'pixelymax', 'classname']

  df = pd.read_csv(appendix_for_annotation_file + 'train.csv', sep=',', header=None, names = columns)
  df.pixelxmax[df.classname == 'empty'] = str()
  df.pixelymin[df.classname == 'empty'] = str()
  df.pixelymax[df.classname == 'empty'] = str()
  df.pixelxmin[df.classname == 'empty'] = str()
  df.classname[df.classname == 'empty'] = str()
  df.to_csv(appendix_for_annotation_file + 'train.csv', header = None, index = None, sep = ',', columns = columns)



  df = pd.read_csv(appendix_for_annotation_file + 'test.csv', sep=',', header=None, names = columns)
  df.pixelxmax[df.classname == 'empty'] = str()
  df.pixelymin[df.classname == 'empty'] = str()
  df.pixelymax[df.classname == 'empty'] = str()
  df.pixelxmin[df.classname == 'empty'] = str()
  df.classname[df.classname == 'empty'] = str()
  df.to_csv(appendix_for_annotation_file + 'test.csv', header = None, index = None, sep = ',', columns = columns)


Working on:  ./stone_dataset/Annotations/v2pickedstone.sqlite stone v2pickedstone
Working on:  ./stone_dataset/Annotations/v2empty.sqlite empty v2empty


/home/peter/anaconda2/envs/GIS/lib/python3.6/site-packages/ipykernel_launcher.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/peter/anaconda2/envs/GIS/lib/python3.6/site-packages/ipykernel_launcher.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/peter/anaconda2/envs/GIS/lib/python3.6/site-packages/ipykernel_launcher.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/peter/anaconda2/envs/GIS/lib/python3.6/site-pa

## Super Resolution


Training Instructions:
 - export the high-res model into tiles of 100x100 /see above
 - we use folder ./super_res_dataset/train_HR for that
 - the script below will create the training and validation datasets
 
 - copy fhe files manually to png

In [31]:
%%capture
os.chdir('/home/peter/git/retina_seafloor')
HR_train = '/home/peter/git/SRGAN-tensorflow/result/images/'    #This is used as the origin folder. The other folders are hardwired
HR_valid = './super_res_dataset/valid_HR/'    # folder must end with /
factor = 4

def img_convert(image, image_format = '.png'):
    #from skimage.io import imread 
    #from skimage.io import imsave
    #img = imread(image, as_gray=True)
    #imsave(image + image_format, img)
    from PIL import Image
    try:
        img = Image.open(image)
        img.save(image + image_format)
    except:
        print('no Image')
    return

def downsampling(folder,image_name, factor, target_folder):
    from skimage.transform import rescale
    from skimage.io import imsave
    from skimage.io import imread
    
    image = folder + image_name
    img = imread(image, as_gray=True)
    img_down = rescale(img, 1/factor)
    #img_up = rescale(img_down, factor)
    imsave(target_folder + image_name, img_down)
    return 
    

def random_distribution(source_folder, target_folder, valid_percentage = 0.2):
    import os
    import random
    import shutil

    filelist = os.listdir(source_folder)  # We assume this is the HR_train folder
    
    number_of_files = len(filelist)
    print(number_of_files)
    
    index_of_files =  range(0,number_of_files-1)
    print(index_of_files)
    
    number_of_valid_files = int(number_of_files * valid_percentage)
    
    random_files_index = random.sample(index_of_files, number_of_valid_files)
    for i in random_files_index:
        shutil.move(source_folder + filelist[i], target_folder + filelist[i])
    return



#convert to png
filelist = os.listdir(HR_train)
print(filelist[0])
for image in filelist:
    img_convert(HR_train + image, '.tif')
    #command = "rm " + HR_train + image
    #os.system(command)

    

#distrubte randomly to validate
#random_distribution(HR_train, HR_valid)


# downsample
#filelist = os.listdir(HR_train)
#for image in filelist:
#    downsampling(HR_train, image, factor, './super_res_dataset/train_LR/' )

#filelist = os.listdir(HR_valid)
#for image in filelist:
#    downsampling(HR_valid, image, factor,'./super_res_dataset/valid_LR_x4/' )


In [ ]:
# Enhance! This!
os.chdir('/home/peter/git/SRGAN-tensorflow')
command = 'sh test_SRResnet.sh'
os.system(command)

In [ ]:
# Computer, transferiere di

In [ ]:
# conver to tif

#make sure its gray
for i in *.png; do convert "$i  -set colorspace Gray -separate -average " "${i%.*}.png"; done


In [ ]:
#generate new world files from low resolution data (see above) with scale factor applied an copy in results image folder

# Merge da tile
gdal_merge.py -o test.tif  *.tif
